In [ ]:
import sqlite3

conn = sqlite3.connect('/tmp/chembl_35/chembl_35_sqlite/chembl_35.db')
cursor = conn.cursor()

In [ ]:
import pandas as pd

from boilerplate import list_tables, get_table_schema, sample, print_size

In [ ]:
from boilerplate import TaxIdMapper

taxid_mapper = TaxIdMapper()

In [ ]:
query = f"SELECT assay_id, assay_type, assay_tax_id, tid FROM assays WHERE assay_type = 'B'"
assays = pd.read_sql_query(query, conn)

print(f'{len(assays)} biochem assays for {len(assays.assay_tax_id.unique())} taxa')

In [ ]:
query = f"SELECT activity_id, assay_id, molregno, standard_type, standard_units, standard_value FROM activities"
activities = pd.read_sql_query(query, conn)

print(f'{len(activities)} activities for {len(assays.assay_id.unique())} assays')

In [ ]:
from boilerplate import molar_scale
import numpy as np

activities['unit_scale'] = activities.standard_units.apply(molar_scale)
activities['pkD'] = - (activities.loc[activities.standard_type == 'Kd'].unit_scale * activities.loc[activities.standard_type == 'Kd'].standard_value).apply(np.log10)

In [ ]:
taxid2rank = {}

with open("/tmp/nodes.dmp") as f:
    for line in f:
        parts = [p.strip() for p in line.split("|")]
        taxid2rank[int(parts[0])] = parts[2]

In [ ]:
assay_id2tax_id = assays.loc[~assays.assay_tax_id.isna()].set_index('assay_id').assay_tax_id.fillna(-1).astype(int).to_dict()
activities['assay_tax_id'] = activities.assay_id.map( assay_id2tax_id )
activities['assay_tax_id'] = activities['assay_tax_id'].fillna(-1).astype(int)

In [ ]:
parent_ids = {'human': 9606,
             'bacteria': 2,
             'virus': 10239,
             'fungi': 4751,
             'Apicomplexa': 5794, #Apicomplexa Plasmodium
             'Euglenozoa': 33682, #Euglenozoa (e.g. Trypanosoma, Leishmania)
             'Amoebozoa': 554915, # Amoebozoa (e.g. Entamoeba)
             'Metamonada': 2611341 # Metamonada (e.g. Giardia)
            }

taxid_mapper.target_ids = set(parent_ids.values())

In [ ]:
activities['parent'] = activities.assay_tax_id.apply(taxid_mapper.get_ancestor_in_set)

In [ ]:
activities['parent'].unique()

In [ ]:
def get_name(parent):
    if parent in (5794, 33682, 554915, 2611341):
        return 'protozoa'
    else:
        for name, tax_id in parent_ids.items():
            if tax_id == parent:
                return name
    return 'unknown'

activities['parent_name'] = activities.parent.apply(get_name)

In [ ]:
sub = activities.loc[(activities.parent_name != 'unknown') & (activities.standard_type == 'Kd')]

In [ ]:
import plotly.express as px
import numpy as np
fig = \
px.violin(sub, x='pkD',
              y='parent_name',
              #box=True,
             template='plotly_white',
             title='Distribution of KD activities across clades',
             category_orders={'parent_name': ['human',
                                              'protozoa',
                                              'fungi',
                                              'bacteria',
                                              'virus',
                                             ]},
             labels={'parent_name': 'group',},)
fig.update_traces(scalemode='count')
fig.show()
# facet_col

In [ ]:
fig = \
px.pie(sub.drop_duplicates(['molregno', 'assay_tax_id']),'parent_name',
              #box=True,
             template='plotly_white',
             title='Number of KD assay points (one per compound / tax-id)',
             category_orders={'parent_name': ['human',
                                              'fungi',
                                              'protozoa',
                                              'bacteria',
                                              'virus',
                                             ]},
             labels={'parent_name': 'group',},
      )
fig.update_traces(hoverinfo='label+percent', textinfo='value')
fig.show()

In [ ]:
sub = activities.loc[(activities.parent_name != 'unknown')]

fig = \
px.pie(sub.drop_duplicates(['molregno', 'assay_tax_id']),'parent_name',
              #box=True,
             template='plotly_white',
             title='Number of assay points (one per compound / tax-id)',
             category_orders={'parent_name': ['human',
                                              'fungi',
                                              'protozoa',
                                              'bacteria',
                                              'virus',
                                             ]},
             labels={'parent_name': 'group',},
      )
fig.update_traces(hoverinfo='label+percent', textinfo='value')
fig.show()

In [ ]:
summary = \
activities.groupby(['standard_type', 'parent_name'])\
          .size().unstack(fill_value=0)\
          .loc[activities.standard_type.value_counts().head(20).index]\
          [['human', 'fungi', 'protozoa', 'bacteria', 'virus', 'unknown' ]]
summary.style.format("{:,}")

In [ ]:
print(summary.to_markdown())

In [ ]:
activities.loc[ activities.parent_name == 'virus'].to_csv('viral_activities.csv')